In [1]:
import os
import random
import time
import copy
from tqdm.notebook import tqdm
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms,models
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from efficientnet_pytorch import EfficientNet

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
print(torch.cuda.get_device_name(torch.cuda.current_device()))

cuda:0
Tesla V100-SXM2-32GB


In [2]:
train_df = pd.read_csv("../input/data/train/labeling_train.csv")
# test_image_path = glob('../input/data/eval/images/*')
train_df = train_df.sort_values("id")

In [3]:
train_df["age_group"] = train_df["age"].apply(lambda x: 0 if x < 30 else 1 if 30 <= x < 58 else 2)
train_df["label"] = train_df["mask"]*6 + train_df["gender"]*3 + train_df["age_group"]

In [4]:
transform = transforms.Compose([
    transforms.Resize((512, 384), Image.BILINEAR),
    transforms.CenterCrop((320,256)),
    transforms.ColorJitter(0.3,0.3,0.3,0.3),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

In [5]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform):
        self.X = dataframe["image_path"]
        self.y = dataframe["label"]
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.X.iloc[index])
        label = self.y.iloc[index]

        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label)

    def __len__(self):
        return len(self.X)

In [6]:
def train_model(model, dataloaders,criterion,optimizer,scheduler,num_epochs):
    start_time = time.time()
    max_accuracy = 0
    min_loss = int(1e9)
    early_stop_point = 0 
    early_stop_limit = 2
    stop_epoch = True
    best_model_wts = copy.deepcopy(model.state_dict())
    history = {"train_loss":[],"train_acc":[],"test_loss":[],"test_acc":[]}
    
    
    for epoch in range(num_epochs):
        f1_list = []
        for phase in ['train', 'test']:
                if phase == 'train':
                    model.train() 
                else:
                    model.eval()   

                running_loss = 0.0
                running_acc = 0.0
                
                # 데이터 반복
                for step, (inputs, labels) in enumerate(tqdm(dataloaders[phase])):
                    inputs, labels = inputs.to(device), labels.to(device)

                    # 매개변수 경사도 0으로 초기화
                    optimizer.zero_grad()

                    # 순전파
                    # train에서만 연산 기록 추적
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)
        
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()
                        
                    running_loss += loss.item() * inputs.size(0)
                    running_acc += torch.sum(preds == labels.data)
                    f1_list.append(f1_score(labels.cpu(),preds.cpu(),average="macro"))
                    
                if phase == "train":
                    scheduler.step()

                epoch_loss = running_loss / len(dataloaders[phase].dataset)
                epoch_acc = running_acc / len(dataloaders[phase].dataset)
                
                print(f"[Epoch : %d] [Phase : %s] epoch_loss : %.5f epoch_acc: %.2f" % 
                      (epoch+1, phase, epoch_loss, epoch_acc))
                
                if phase == "train":
                    history["train_loss"].append(epoch_loss)
                    history["train_acc"].append(epoch_acc)
                else:
                    history["test_loss"].append(epoch_loss)
                    history["test_acc"].append(epoch_acc)
                            
                if phase == "test" and max_accuracy <= epoch_acc:
                    max_accuracy = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    early_stop_point = 0
                    
                elif phase == "test" and max_accuracy > epoch_acc:
                    early_stop_point += 1
                    
                if phase == "test" and min_loss > epoch_loss:
                    min_loss = epoch_loss
                    
                if early_stop_point == early_stop_limit:
                    print("early_stopped")
                    stop_epoch = False
                    break
                
        print(f"[Epoch : %d] [Phase : %s] f1_score : %.5f" % 
                      (epoch+1, phase, np.sum(f1_list) / len(f1_list)))
        if stop_epoch == False:
            break
    print(time.time()-start_time)
    model.load_state_dict(best_model_wts)
    print("Done")
    return model,history

In [7]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
foldperf={}

for fold, (train_idx,test_idx) in enumerate(skf.split(train_df["id"], train_df["label"])):
    print('Fold {}'.format(fold + 1))
    
    train = train_df.iloc[train_idx]
    test = train_df.iloc[test_idx]
    
    train_data = CustomDataset(train,transform)
    eval_data = CustomDataset(test,transform)
    
    train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=256, shuffle=True, num_workers=1)
    eval_dataloader = torch.utils.data.DataLoader(eval_data, batch_size=256, shuffle=True, num_workers=1)
    
    # pretrained model 불러오기
    model = torchvision.models.resnet18(pretrained=True)
    
    num_classes = 18 # 데이터에 맞게 class 개수 변경
    fc_input = model.fc.in_features # fc layer의 input 수
    model.fc = nn.Linear(fc_input, num_classes) # fc layer 입출력 변경

    torch.nn.init.kaiming_uniform_(model.fc.weight) # he init으로 fc layer weight, bias 초기화
    stdv = 1/(fc_input**(1/2)) 
    model.fc.bias.data.uniform_(-stdv,stdv)

    model.to(device) # GPU에 올리기

    dataloaders = {
        "train" : train_dataloader,
        "test" : eval_dataloader
    }
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    
    model_ft,history = train_model(model, dataloaders,criterion, optimizer, scheduler,num_epochs=10)
    foldperf["fold{}".format(fold+1)] = history
torch.save(model_ft.state_dict(),"./kfold_restnet.pt")

Fold 1



[Epoch : 1] [Phase : train] epoch_loss : 0.87798 epoch_acc: 0.72



[Epoch : 1] [Phase : test] epoch_loss : 0.40894 epoch_acc: 0.84
[Epoch : 1] [Phase : test] f1_score : 0.57866



[Epoch : 2] [Phase : train] epoch_loss : 0.29387 epoch_acc: 0.89



[Epoch : 2] [Phase : test] epoch_loss : 0.33355 epoch_acc: 0.87
[Epoch : 2] [Phase : test] f1_score : 0.78626


...


[Epoch : 8] [Phase : test] epoch_loss : 0.19636 epoch_acc: 0.93
[Epoch : 8] [Phase : test] f1_score : 0.95083



[Epoch : 9] [Phase : train] epoch_loss : 0.06570 epoch_acc: 0.99



[Epoch : 9] [Phase : test] epoch_loss : 0.19697 epoch_acc: 0.93
[Epoch : 9] [Phase : test] f1_score : 0.95251



[Epoch : 10] [Phase : train] epoch_loss : 0.06665 epoch_acc: 0.99



[Epoch : 10] [Phase : test] epoch_loss : 0.19667 epoch_acc: 0.93
[Epoch : 10] [Phase : test] f1_score : 0.95296
2907.0775768756866
Done


In [10]:
for j in range(1,6):
    for i in range(len(foldperf["fold"+str(j)]["train_acc"])):
        foldperf["fold"+str(j)]["train_acc"][i] = foldperf["fold"+str(j)]["train_acc"][i].cpu().numpy()
        foldperf["fold"+str(j)]["test_acc"][i] = foldperf["fold"+str(j)]["test_acc"][i].cpu().numpy()

In [11]:
testl_f,tl_f,testa_f,ta_f=[],[],[],[]
k=5
for f in range(0,5):
    tl_f.append(np.mean(foldperf['fold{}'.format(f+1)]['train_loss']))
    testl_f.append(np.mean(foldperf['fold{}'.format(f+1)]['test_loss']))
    
    ta_f.append(np.mean(foldperf['fold{}'.format(f+1)]['train_acc']))
    testa_f.append(np.mean(foldperf['fold{}'.format(f+1)]['test_acc']))

print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.3f} \t Average Test Loss: {:.3f} \t Average Training Acc: {:.2f} \t Average Test Acc: {:.2f}".format(np.mean(tl_f),np.mean(testl_f),np.mean(ta_f),np.mean(testa_f)))     

Performance of 5 fold cross validation
Average Training Loss: 0.204 	 Average Test Loss: 0.244 	 Average Training Acc: 0.94 	 Average Test Acc: 0.91


In [100]:
print('Performance of {} fold cross validation'.format(k))
print("Average Training Loss: {:.3f} \t Average Test Loss: {:.3f} \t Average Training Acc: {:.2f} \t Average Test Acc: {:.2f}".format(np.mean(tl_f),np.mean(testl_f),np.mean(ta_f),np.mean(testa_f)))     

Performance of 5 fold cross validation
Average Training Loss: 0.844 	 Average Test Loss: 2.428 	 Average Training Acc: 0.73 	 Average Test Acc: 0.54
